In [1]:
import pandas as pd
import numpy as np
import scipy.sparse as sp
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_distances
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from scipy.sparse.linalg import svds

pd.options.display.max_colwidth = 2000
pd.options.display.float_format = '{:.2f}'.format

In [2]:
ratings = pd.read_csv('ratings_df_sample_2.csv')
movies = pd.read_csv('movies.csv')

Предположим, постановка рейтинга — обязательное по итогам просмотра фильмов действие. Основываясь на этом, сгенерируйте новый целевой признак «факт просмотра фильма пользователем», который будет равен 1 для всех пар пользователь * фильм из подгруженного датасета.

In [3]:
ratings['watched'] = 1

Сначала найдите уникальные id всех пользователей и уникальные id всех фильмов.

С помощью функции random.choice (документация) сгенерируйте случайные пары пользователь * фильм

Поскольку среди сгенерированных пар могут быть и такие, что пользователь в них уже смотрел фильм, сгенерируйте побольше пар, например, удвоенное количество строк из источника. Это может занять пару минут.

Среди сгенерированных пар могут быть и дубликаты, удалите их.

Для удобства отмасштабируем идентификаторы фильмов таким образом, чтобы они начинались с 0 и заканчивались на n_movies-1
Этот метод позволит находить фильм по индексу матрицы (i-й столбец матрицы это i-й фильм)

In [4]:
%%time
movies_values = ratings['movieId'].unique()

ratings['movieId'] = ratings['movieId'].apply(lambda f: np.where(movies_values == f)[0][0])

CPU times: total: 46.4 s
Wall time: 46.4 s


Также отмасштабируем идентификаторы пользователей таким образом, чтобы они начинались с 0 и заканчивались на n_users-1
Этот метод позволит находить пользовтеля по индексу матрицы (i-я строка матрицы это i-й пользователь)

In [5]:
%%time
users_values = ratings['userId'].unique()

ratings['userId'] = ratings['userId'].apply(lambda f: np.where(users_values == f)[0][0])

CPU times: total: 2min 12s
Wall time: 2min 13s


In [6]:
ratings

,userId,movieId,rating,timestamp,watched
0,0,0,3.00,974918176,1
1,0,1,5.00,974836809,1
2,0,2,4.00,974837760,1
3,0,3,4.00,974837760,1
4,0,4,5.00,974840217,1
...,...,...,...,...,...
6040094,19999,841,3.50,1312929587,1
6040095,19999,993,3.50,1311624755,1
6040096,19999,934,2.00,1271506505,1
6040097,19999,842,2.00,1276962374,1


In [7]:
n_users = ratings['userId'].nunique()
n_movies = ratings['movieId'].nunique()
(n_users, n_movies)

(20000, 1000)

In [8]:
unique_users = ratings['userId'].unique()
unique_movies = ratings['movieId'].unique()

In [9]:
# Количество пар, которые мы хотим сгенерировать (удвоенное количество строк источника)
num_pairs = 2 * len(ratings)

# Случайные пары пользователь * фильм
generated_pairs = pd.DataFrame({
    'userId': np.random.choice(unique_users, num_pairs),
    'movieId': np.random.choice(unique_movies, num_pairs)
})

In [10]:
# Удалим дубликаты
generated_pairs = generated_pairs.drop_duplicates()

In [11]:
generated_pairs

,userId,movieId
0,1890,705
1,11705,790
2,13838,236
3,2082,923
4,9514,927
...,...,...
12080189,13066,983
12080190,5478,776
12080191,9656,922
12080192,6405,22


Оставьте среди сгенерированных пар только те, в которых пользователь фильм не смотрел.

Возможно, пар получилось больше, чем нужно, выберите из них столько, сколько у нас строк в исходных данных.

Добавьте очищенные сгенерированные пары к исходным данным. Значение целевого признака в них будет равно нулю. Убедитесь, что у вас не появились дубликаты в датасете.

In [12]:
pairs_watched = ratings[['userId', 'movieId']]
pairs_watched

,userId,movieId
0,0,0
1,0,1
2,0,2
3,0,3
4,0,4
...,...,...
6040094,19999,841
6040095,19999,993
6040096,19999,934
6040097,19999,842


In [13]:
# Преобразование столбцов в множества
set1 = set(zip(generated_pairs['userId'], generated_pairs['movieId']))
set2 = set(zip(pairs_watched['userId'], pairs_watched['movieId']))

# Находим различающиеся элементы
difference_set = set1.difference(set2)

# Преобразование результата обратно в датафрейм
pairs_not_watched = pd.DataFrame(list(difference_set), columns=['userId', 'movieId'])

In [14]:
pairs_not_watched

,userId,movieId
0,12833,705
1,2236,469
2,10111,899
3,4908,188
4,10059,709
...,...,...
6331886,18433,887
6331887,17344,923
6331888,9469,493
6331889,18331,610


In [15]:
pairs_not_watched = pairs_not_watched.sample(n=ratings.shape[0])

In [16]:
ratings_new = pd.concat([ratings, pairs_not_watched], ignore_index=True)

In [17]:
ratings_new[['userId', 'movieId']].duplicated().sum()

0

In [18]:
ratings_new = ratings_new.fillna(0)
ratings_new.sample(5)

,userId,movieId,rating,timestamp,watched
1133206,3287,198,3.00,973826445.00,1.00
1983751,5710,550,3.50,1148581932.00,1.00
11261387,1127,772,0.00,0.00,0.00
5612214,17917,593,5.00,1100357168.00,1.00
10015266,18228,321,0.00,0.00,0.00


In [19]:
X = ratings_new.iloc[:, :-1]
y = ratings_new.iloc[:, -1]

Подготовьте датасет к обучению: отделите тестовую часть от тренировочной.

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Обучите dummy-model. Пусть она будет возвращать случайную вероятность принадлежности классу 1. Для этого можете использовать функцию random.random (документация). Оцените ее качество какой-то метрикой на свой вкус. Необходимо прогнозировать именно вероятность, чтобы была возможность ранжировать по ней варианты для рекомендации лучшего контента пользователю.

In [21]:
import random

def dummy_model(X):
    # Генерируем случайные вероятности для класса 1
    probabilities = [random.random() for _ in range(len(X))]

    return probabilities

In [22]:
dummy_pred = dummy_model(X_test)

In [23]:
from sklearn.metrics import f1_score

rmse = np.sqrt(mean_squared_error(y_test, dummy_pred))
f1 = f1_score(y_test, np.round(dummy_pred))

print(f"RMSE: {rmse}")
print(f"F1 Score: {f1}")

RMSE: 0.5774406746313889
F1 Score: 0.4998596595232398


Реализуйте три алгоритма коллаборативной фильтрации: user-, item-based и алгоритм на основе матричной факторизации. Оцените их качество и адекватность. Если качество недостаточно хорошее, попробуйте варьировать параметры: количество похожих пользователей/фильмов, количество элементов в матрицах при матричном разложении.

Для оценки прогнозов с помощью колаборативной фильтрации, разобьем датасет на train и test

In [26]:
train_data, test_data = train_test_split(ratings_new, test_size=0.01)

print('Train shape: {}'.format(train_data.shape))
print('Test shape: {}'.format(test_data.shape))

Train shape: (11959396, 5)
Test shape: (120802, 5)


Сформируем матрицу user-item

In [27]:
n_users = train_data['userId'].nunique()
n_movies = train_data['movieId'].nunique()
(n_users, n_movies)

(20000, 1000)

Создаём user-item матрицу – для обучения

In [28]:
%%time
train_data_matrix = np.zeros((n_users, n_movies))
for line in train_data.to_dict(orient='records'):
    train_data_matrix[line['userId'], line['movieId']] = line['watched']  

CPU times: total: 57.1 s
Wall time: 1min 7s


In [29]:
train_data_matrix.shape

(20000, 1000)

In [30]:
train_data_matrix

array([[1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

Матрицы попарных косинусных расстояний

In [31]:
%%time
# считаем попарное косинусное расстояние для пользователей (строк матрицы)
user_similarity = cosine_distances(train_data_matrix)

CPU times: total: 33.8 s
Wall time: 25.2 s


In [32]:
# # считаем попарное косинусное расстояние для фильмов (столбцов матрицы)
movie_similarity = cosine_distances(train_data_matrix.T)

**user-based**

Для каждого пользователя находим топ 10 ближайших соседей, исключая себя самого (поэтому индекс от единицы до top + 1)

In [33]:
%%time
top=10
top_similar_users = []
for i in range(n_users):
    neighbors = (user_similarity[i]).argsort()[1:top + 1]
    top_similar_users.append(
        train_data_matrix[neighbors]
    )
top_similar_users = np.array(top_similar_users)

CPU times: total: 39.8 s
Wall time: 47.8 s


Количество  пользователей, количество соседей, количество фильмов

In [34]:
top_similar_users.shape

(20000, 10, 1000)

Для каждого пользователя и каждого фильма считаем средний факт просмотра у фильма от топ похожих пользователей

Так делам матрицу с предсказанными просмотрами

In [35]:
predicted_watched_user_based = top_similar_users.mean(1)
predicted_watched_user_based.shape

(20000, 1000)

Делаем предикт на тестовом датасете, для каждого тестового userId и movieId заполняем предсказанный факт просмотра из полученной матрицы

По userId строке и movieId столбцу, для этого и делали смену айдишников

In [38]:
test_data


,userId,movieId,rating,timestamp,watched
5933172,19390,938,4.00,974655414.00,1.00
209835,628,388,2.00,942253726.00,1.00
901283,2613,53,4.00,1219694830.00,1.00
3792652,11138,888,3.00,1035298472.00,1.00
7547130,19833,494,0.00,0.00,0.00
...,...,...,...,...,...
4452172,13506,250,4.50,1135595185.00,1.00
9949967,10257,554,0.00,0.00,0.00
7180762,19399,474,0.00,0.00,0.00
11499164,18213,106,0.00,0.00,0.00


In [40]:
test_data['predict_user_based'] = test_data.apply(
    lambda f: round(predicted_watched_user_based[int(f['userId'].astype(int)), int(f['movieId'].astype(int))]), axis=1
)

In [52]:
rmse = np.sqrt(mean_squared_error(test_data['predict_user_based'], test_data['watched']))
f1 = f1_score(test_data['watched'], np.round(test_data['predict_user_based']))

print(f"RMSE: {rmse}")
print(f"F1 Score: {f1}")

RMSE: 0.4686580878471864
F1 Score: 0.7883202361482309


**item-based**

Для каждого фильма находим топ 10 ближайших соседей, исключая себя самого (поэтмоу индекс от единицы до top + 1)

In [54]:
top=10
top_similar_movies = []
for i in range(n_movies):
    neighbors = (movie_similarity[i]).argsort()[1:top + 1]
    top_similar_movies.append(
        train_data_matrix.T[neighbors]
    )
    
top_similar_movies = np.array(top_similar_movies)

Число фильмов, число соседей, число пользователей

In [55]:
top_similar_movies.shape

(1000, 10, 20000)

Для каждого пользователя и каждого фильма считаем средний факт просмотра от пользователя для топ соседних фильмов

In [56]:
predicted_watched_item_based = top_similar_movies.mean(1).T
predicted_watched_item_based.shape

(20000, 1000)

In [85]:
test_data['predict_item_based'] = test_data.apply(
    lambda f: round(predicted_watched_item_based[int(f['userId'].astype(int)), int(f['movieId'].astype(int))]), axis=1
)

Делаем предикт на тестовом датасете, для каждого тестового userId и movieId заполняем предсказанный факт просмотра из полученной матрицы

По userId строке и movieId столбцу, для этого и делали смену айдишников

test_data['predict_item_based'] = test_data.apply(
    lambda f: round(predicted_watched_item_based[int(f['userId'].astype(int)), int(f['movieId'].astype(int))]), axis=1
)

In [86]:
rmse = np.sqrt(mean_squared_error(test_data['predict_item_based'], test_data['watched']))
f1 = f1_score(test_data['watched'], np.round(test_data['predict_item_based']))

print(f"RMSE: {rmse}")
print(f"F1 Score: {f1}")

RMSE: 0.4887435087860574
F1 Score: 0.777386903659816


**Алгоритм на основе матричного разложения**

In [62]:
%%time
# делаем SVD
u, s, vh = svds(train_data_matrix, k=20)
s_diag_matrix = np.diag(s)

users = np.dot(u, s_diag_matrix)
items = vh.T

CPU times: total: 8.55 s
Wall time: 3.19 s


In [90]:
def custom_round(value):
    return 1 if (round(value) != 0) else 0

In [91]:
test_data['svd_predictions'] = test_data.apply(
    lambda f: int(custom_round(np.dot(users[int(f['userId'])], items[int(f['movieId'])]))), axis=1
)

In [93]:
rmse = np.sqrt(mean_squared_error(test_data['svd_predictions'], test_data['watched']))
f1 = f1_score(test_data['watched'], np.round(test_data['svd_predictions']))

print(f"RMSE: {rmse}")
print(f"F1 Score: {f1}")

RMSE: 0.48593229623058104
F1 Score: 0.7186467426147851


user-based:
- RMSE: 0.4686580878471864;
- F1 Score: 0.7883202361482309

item-based:
- RMSE: 0.4887435087860574;
- F1 Score: 0.777386903659816

Алгоритм на основе матричного разложения:
- RMSE: 0.48593229623058104;
- F1 Score: 0.7186467426147851

User-based алгоритм дает наименьшую среднеквадратическую ошибку (RMSE), что является хорошим показателем точности предсказаний.
User-based алгоритм также имеет наивысший F1 Score, что указывает на хорошее сочетание точности и полноты в бинарной классификации (вероятность того, что пользователь посмотрит фильм).

Вывод:
На основе метрик, можно сказать, что user-based алгоритм проявил себя лучше всего с точки зрения как точности предсказаний (низкое RMSE), так и учета баланса между точностью и полнотой (высокий F1 Score).